In [1]:
import copy

import numpy as np
import pandas as pd
from itertools import combinations

from sklearn.model_selection import KFold
from skopt.space import Real, Categorical, Integer

import sys
sys.path.append('G:\내 드라이브\대학\대외\2023\연구원\연구과제\Ensemble-Kernel')

from Ensemble_Kernel_New import split_data,prepare_response_variable, c_o, nom, new_kernel, train_predict_fastKernelSurvivalSVM_bayesian, train_predict_fastKernelSurvivalSVM_clinical_bayesian, one_hot_encode_columns, remove_high_vif_columns, convert_categorical_to_boolean, new_kernel_no_abs, compare_kernels_with_abs_and_no_abs, train_predict_fastKernelSurvivalSVM_bayesian_no_abs, train_predict_fastKernelSurvivalSVM, train_predict_fastKernelSurvivalSVM_AFT, new_kernel_AFT, train_predict_fastKernelSurvivalSVM_bayesian_AFT, train_predict_fastKernelSurvivalSVM_clinical, train_predict_fastKernelSurvivalSVM_clinical_bayesian, C_index_fastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
from sklearn.model_selection import ShuffleSplit, GridSearchCV

### 전처리

In [2]:
data = pd.read_csv("veteran_data.csv")

df = data.drop(['Unnamed: 0'], axis=1)
df = df.astype({'trt' : 'category', 'celltype' : 'category', 'prior' : 'category'})
df.replace({'celltype' : {'squamous' : 0, 'large' : 1, 'smallcell' : 2, 'adeno' : 3}}, inplace = True)

df = df.rename(columns={'age':'Age',
                        'time':'OS',
                        'status':'Status'})

#onehot encode
df_onehot, categorical_columns = one_hot_encode_columns(df)

columns_to_convert=['celltype_1', 'celltype_2','celltype_0', 'prior_10', 'trt_2']

df_onehot[columns_to_convert] = df_onehot[columns_to_convert].astype('category')

df_onehot

,OS,Status,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2
0,72,1,60,7,69,0,0,1,0,0
1,411,1,70,5,64,0,0,1,1,0
2,228,1,60,3,38,0,0,1,0,0
3,126,1,60,9,63,0,0,1,1,0
4,118,1,70,11,65,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
132,133,1,75,1,65,1,0,0,0,1
133,111,1,60,5,64,1,0,0,0,1
134,231,1,70,18,67,1,0,0,1,1
135,378,1,80,4,65,1,0,0,0,1


In [3]:
#train, test split
x_train, x_test, target_train, target_test = split_data(df_onehot)

y_train = prepare_response_variable(target_train)
y_test = prepare_response_variable(target_test)

#Linear
x_train_linear=x_train.drop(['OS','Status'],axis=1)
x_test_linear=x_test.drop(['OS','Status'],axis=1)

In [4]:
x_train

,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2,OS,Status
9,70,6,70,0,0,1,0,0,100,0
20,40,3,55,0,1,0,0,0,123,0
72,50,8,52,0,0,1,1,1,231,0
71,80,3,48,0,0,1,0,1,87,0
13,80,9,52,0,0,1,1,0,25,0
...,...,...,...,...,...,...,...,...,...,...
116,70,8,61,0,0,0,0,1,36,1
96,20,11,66,0,1,0,0,1,7,1
12,30,4,63,0,0,1,0,0,144,1
7,80,29,68,0,0,1,0,0,110,1


KFold Setting

In [5]:
#KFold setting
cv=KFold(n_splits = 5, shuffle=True, random_state=36)

param_space = {
    'alpha': Real(1e-6, 1e+6, 'log-uniform'),
}
param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}

column 선택 조합 만들기

In [22]:
#os, status 분리
drop = x_train[['Age','OS','Status']]
x_train_drop = x_train.drop(columns=['Age','OS','Status'])

# 데이터프레임의 열 이름을 리스트로 저장
columns = x_train_drop.columns

column_groups = []

for i in range(len(columns)):
    # i+1개씩 선택한 모든 열 조합을 생성
    all_column_combinations = list(combinations(columns, i+1))

    # 각 조합에 대한 데이터프레임을 생성하고 출력
    for column_combination in all_column_combinations:
        selected_columns = list(column_combination)
        sub_train1 = x_train_drop[selected_columns]
        sub_train2 = pd.concat([sub_train1, drop], axis=1)
        column_groups.append(sub_train2)

Cross validation

In [ ]:
num_groups = 3
best_cindex = []

for i in range(len(column_groups)):
    x_groups = []
    train = column_groups[i].sample(frac=1).reset_index(drop=True)

    # train data를 그룹으로 나누기
    for i in range(num_groups):
        censored = train[train['Status'] == 0]  # 0인 행만 추출
        uncensored = train[train['Status'] == 1]  # 1인 행만 추출

        group_size1 = len(censored) // num_groups
        group_size2 = len(uncensored) // num_groups

        if i < num_groups - 1:
            #censoring 비율 맞추기
            group1 = censored.iloc[i * group_size1:(i + 1) * group_size1]
            group2 = uncensored.iloc[i * group_size2:(i + 1) * group_size2]

            group = pd.concat([group1, group2], ignore_index=True)
        else:
            group1 = censored.iloc[i * group_size1:]
            group2 = uncensored.iloc[i * group_size2:]

            group = pd.concat([group1, group2], ignore_index=True)
    
        x_groups.append(group)
    
    cindex=[]

    for i in range(len(x_groups)):

        temp = []
        for j in range(len(x_groups)):
            if i != j:
                temp.append(x_groups[j])
        #train = train data(등분한 것 중 1개를 제외한 나머지)
        train=pd.concat(temp)

        #validation = validation data(등분한 것 중 1개)
        validation = x_groups[i]

        train_target=train[['Status','OS']]
        test_target=validation[['Status','OS']]

        train_y=prepare_response_variable(train_target)
        test_y=prepare_response_variable(test_target)
        #data정의 -> train, validation, train_y, test_y

        result = C_index_fastKernelSurvivalSVM(train, train_y, validation, test_y, param_grid, keywords = ['Age'], drop=False, coef_drop=None)
    
        cindex.append(result[1])
        #print(result[1]) #생략 가능
    
    best_cindex.append(np.mean(cindex))

In [48]:
#best_cindex : 선택된 변수별 모든 validation cindex값을 list로 저장
#max_num : best cindex의 리스트 번호
max_num = best_cindex.index(max(best_cindex))

#train_column : best cindex의 선택된 변수
train_column = column_groups[max_num].columns

#선택된 변수만을 가지고 train, test 재정의
#x_train_re, y_train, x_test_re, y_test
x_train_re=x_train[train_column]
x_test_re=x_test[train_column]

In [49]:
#result_ensemble : train cindex, test cindex, remaining variables 순서대로 결과가 저장된 리스트
result_ensemble = C_index_fastKernelSurvivalSVM(x_train_re, y_train, x_test_re, y_test, param_grid, keywords = ['Age', 'Sex'], drop=False, coef_drop=None)

In [51]:
result_ensemble

(0.7397047397047397,
 0.7225609756097561,
 ['karno', 'diagtime', 'celltype_2', 'trt_2', 'Age'])